# Introduction and Motivation

On March 4, 2020 California's Governor, Gavin Newsom, declared a State of Emergency to prepare for the broader spread of COVID-19 [[source](https://www.gov.ca.gov/2020/03/04/governor-newsom-declares-state-of-emergency-to-help-state-prepare-for-broader-spread-of-covid-19/])]. Prompted by the Governor's State of Emergency, educational policy makers had to quickly decide on whether or not to close schools and for how long to close them for. On March 9, 2020, UC Berkeley made its first campus-wide decision on the pandemic, choosing to suspended insturction for a two week period lasting from March 10 to March 29 [[News.Berkeley](https://news.berkeley.edu/2020/03/09/as-coronavirus-spreads-uc-berkeley-suspends-in-person-instruction/])]. At the time, questions emerged about instruction during the rest of the spring semester, but most assumed that instruction would return to its in-person normal by the Fall 2020 semester. Nevertheless, as the Fall 2020 semester drew closer and closer, campus became increasingly unclear on its decision around in-person instruction. On June 17, 2020, Berkeley announced that it would adopt a hybrid teaching model with some in-person teaching [[source](https://news.berkeley.edu/2020/06/17/fall-2020-information-for-faculty-and-staff/)]. However, on July 21, 2020, Campus changed their decision, deciding to hold the begining the Fall 2020 semester fully remote [[source](https://news.berkeley.edu/2020/06/17/fall-2020-information-for-faculty-and-staff/)]. Campus' lack of a timely and cogent plan for the Fall 2020 semester meant the students could not adequetly adjust or adapt to new circumstances. As such, student populations in precarious housing or economic states were made increasingly vulnerable. If campus administrators had reliable forecasts prior to June 1 on the number of local COVID cases, deaths, and  hospitalizations for the Fall 2020 semester, they could have made a much more cohesive and timely decision on whether or not to hold the Fall 2020 semester in-person. 

California State Government provides a single source database to track COVID cases, deaths, hospitalizations, and ICU patients for 56 of the State's 58 counties [[ca.gov](https://covid19.ca.gov/state-dashboard/)]. The two counties not tracked are the two smallest counties in California as of 2021—Sierra County and Alpine County—with populations of 3,079 and 1,344 respectively [[source](https://www.california-demographics.com/counties_by_population)]. The ca.gov database contains data on COVID cases and deaths starting from February 1, 2020, and hospitalization and ICU data starts from March 29, 2020. 

We explore if in forecasting the COVID-19 landscape for the Fall 2020 semester using sparse data prior to June 1, 2020, machine learning models can produce better predictions than a simple linear fit. 

# Goals

The goal of this project is to develop a machine learning forecasting model that predicts Alameda COVID cases, deaths, positive tests, hospitalizations, and ICU patients for the August 1, 2020 to December 31, 2020 timeframe using input data from ca.gov over the range February 1, 2020 to May 31, 2020. We baseline our machine learning models against a simple linear model. These scenarios and goals put ourselves in the position of UC Berkeley's Administrators in making a sound and timely deicsion on whether or not to hold the Fall 2020 semester in-person or not. 

# Data Source

We are interested in data on COVID-19 cases, deaths, hospitalizations, and ICU admissions. CA.gov reports on [COVID-19 cases and deaths](https://data.chhs.ca.gov/dataset/covid-19-time-series-metrics-by-county-and-state) per day per county from 2/1/2020 to 3/1/2023. This repository contains the following relevant columns:

1. **Date**
2. **County**
3. **Population**: county population per the 2020 CA Department of Finance estimations.
4. **Cases**: laboratory-confirmed COVID-19 cases with episode date equal to the row-wise date.
5. **Deaths**: confirmed by local health departments.
6. **Tests**: laboratory conducted tests with collection date equal to the row-wise date.
7. **Positive Tests**: positive PCR tests with collection date equal to the row-wise date.

CA.gov also reports on [COVID-19 hospitalizations and ICU patients](https://data.ca.gov/dataset/covid-19-hospital-data1) per day per county from 3/29/2020 to 3/1/2023. The relevant columns are:

1. **Date**
2. **County**
3. **Hospitalizations**: number of patients hospitalized in an inpatient bed who have laboratory-confirmed COVID-19).
4. **Suspected Hospitalizations**: number of patients hospitalized in an inpatient bed without a laboratory-confirmed COVID-19 diagnosis, but who have signs and symptoms compatible with COVID-19 specified by the CDC's [Interim Public Health Guidance for Evaluating Persons Under Investigation](https://stacks.cdc.gov/view/cdc/85306)).
5. **Hospital Beds**: total number of beds in facilities across the county (including surge beds, inpatient beds, outpatient post-surgival beds, labor and delivery unit beds, and observation beds). 
6. **ICU Patients**: total number of laboratory-confirmed positive COVID-19 patients that are in any ICU unit (incl. NICU, PICU, and adult ICU). 
7. **Available ICU Beds**: total number of ICU beds available (incl. NICU, PICU, and adult ICU). 

Since our goal is to simmulate the decision making process for UC Berkeley campus administrators leading up the Fall semester, we remove all data after 2020. Model Training and validation is conducted on data before 6/1/2020 (i.e., values in the range 2/1/2020-5/31/2023 for the cases/deaths data and values in the range 3/29/2020-5/31/2023 for the hospitalization data). Final model performance is evaluated on data around the Fall semester, from 8/1/2020 to 12/31/2020. 

# Features

For ease of notation, we define the following variables that correspond directly to values in our data. 

1. $\mathcal{C}_t$: cases at time $t$.
2. $\mathcal{D}_t$: deaths at time $t$.
3. $\mathcal{P}_t$: positive tests at time $t$.
4. $\mathcal{P}'_t$: tests at time $t$.
5. $\mathcal{H}_t$: hospitalizations at time $t$.
6. $\mathcal{H}'_t$: hospital beds at time $t$.
7. $\mathcal{H}''_t$: suspected hospitalizations at time $t$.
8. $\mathcal{I}_t$: ICU patients at time $t$.
9. $\mathcal{I}'_t$: ICU beds at time $t$.

Where $T$ is the set of all dates the model is trained on and $t\in T$. We omit the subscript $t$ to refer to each variable's time series. For example, $\mathcal{C} \equiv \{\mathcal{C}_t | t \in T\}$. <span style="color: red;"> The features of our model are lagged values of $\mathcal{C}, \mathcal{D}, \mathcal{P}, \mathcal{P}', \mathcal{H}, \mathcal{H}', \mathcal{H}'', \mathcal{I},$ and $\mathcal{I}'$. The outputs of our model are point estimates $\mathcal{C}_{t'}, \mathcal{D}_{t'}, \mathcal{P}_{t'}, \mathcal{H}_{t'},$ and $\mathcal{I}_{t'}$ where $t' \succ \text{sup} \space T$ </span>. Thus, our model $f|t':\mathbb{R}^{|T| \space \times \space 9} \rightarrow \mathbb{R}^5$ is, 

$$f(t', \mathcal{C}_t, \mathcal{D}_t, \mathcal{T}_t, \mathcal{T}'_t, \mathcal{H}_t, \mathcal{H}'_t, \mathcal{H}''_t, \mathcal{I}_t, \mathcal{I}''_t) = \{\mathcal{C}_{t'}, \mathcal{D}_{t'}, \mathcal{P}_{t'}, \mathcal{H}_{t'}, \mathcal{I}_{t'}\}.$$ 